In [1]:
import socket
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from os import urandom
from cryptography.fernet import Fernet

In [2]:
def start_client():
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(('localhost', 9090))
        
        while True:
            data = input()
            client_socket.sendall(data.encode())
            response = client_socket.recv(1024)
            
            if response.decode('utf-8') == 'exit':
                print('Клиентский сокет закрыт')
                break
                
            if response.decode('utf-8') == 'shutdown':
                print('Сервер и клиентский сокет закрыты')
                break
                
            print(f"Ответ от сервера: {response.decode('utf-8')}")
            
    except ConnectionRefusedError:
        print('Невозможно подключиться к серверу')

    finally:
        client_socket.close()

In [9]:
start_client()

dvdv
Ответ от сервера: dvdv
exit
Клиентский сокет закрыт


# Лабораторная работа №8

In [11]:
def start_client():
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(('localhost', 9090))
        
        params_bytes = client_socket.recv(1024)
        parameters = serialization.load_pem_parameters(params_bytes, backend=default_backend())
        
        private_key = parameters.generate_private_key()
        public_key = private_key.public_key()
        
        public_key_b = public_key.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo
        )
        client_socket.sendall(public_key_b)
        
        server_public_key_b = client_socket.recv(1024)
        server_public_key = serialization.load_pem_public_key(server_public_key_b)
        
        shared_secret = private_key.exchange(server_public_key)
        derived_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info='handshake data'.encode(),
            backend=default_backend()
        ).derive(shared_secret)
        
        while True:
            data = input().encode('utf-8')
            iv = urandom(16)
            cipher = Cipher(algorithms.AES(derived_key), modes.CFB(iv), backend=default_backend())
            encryptor = cipher.encryptor()
            ciphertext = encryptor.update(data) + encryptor.finalize()
            
            client_socket.sendall(iv)
            client_socket.sendall(ciphertext)
            
            iv = client_socket.recv(1024)
            cipher_response = client_socket.recv(1024)

            cipher = Cipher(algorithms.AES(derived_key), modes.CFB(iv), backend=default_backend())
            decryptor = cipher.decryptor()
            response = decryptor.update(cipher_response) + decryptor.finalize()
            
            if response.decode('utf-8') == 'exit':
                print('Клиентский сокет закрыт')
                break
                
            if response.decode('utf-8') == 'shutdown':
                print('Сервер и клиентский сокет закрыты')
                break
                
            print(f"Ответ от сервера: {response.decode('utf-8')}")
            
    except ConnectionRefusedError:
        print('Невозможно подключиться к серверу')

    finally:
        client_socket.close()

In [22]:
start_client()

jj
Ответ от сервера: jj
shutdown
Сервер и клиентский сокет закрыты


In [3]:
def start_client():
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(('localhost', 9090))
        
        key = client_socket.recv(1024)
        cipher = Fernet(key)
        while True:
            data = input().encode()
            ciphertext = cipher.encrypt(data)
            client_socket.sendall(ciphertext)
            
            cipher_response = client_socket.recv(1024)
            response = cipher.decrypt(cipher_response)
            
            if response.decode('utf-8') == 'exit':
                print('Клиентский сокет закрыт')
                break
                
            if response.decode('utf-8') == 'shutdown':
                print('Сервер и клиентский сокет закрыты')
                break
                
            print(f"Ответ от сервера: {response.decode('utf-8')}")
            
    except ConnectionRefusedError:
        print('Невозможно подключиться к серверу')

    finally:
        client_socket.close()

In [4]:
start_client()

jdhfjdhf
Ответ от сервера: jdhfjdhf
kjkdc
Ответ от сервера: kjkdc
exit
Клиентский сокет закрыт
